In [2]:
import pandas as pd
import numpy as np
import glob
import sys
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
py_file_location = "/content/drive/MyDrive"
sys.path.append(os.path.abspath(py_file_location))

In [5]:
from function_tools import *

In [6]:
path_files = r"/content/drive/MyDrive/Data_M/1M_Lem_clusters/"

In [7]:
# récupérer les clusters 
# glob aide à faire un trie par regex afin d'en choisir les fichiers nettoyés
list_file= glob.glob(f"{path_files}*")

In [8]:
list_file

['/content/drive/MyDrive/Data_M/1M_Lem_clusters/km_50',
 '/content/drive/MyDrive/Data_M/1M_Lem_clusters/som_400',
 '/content/drive/MyDrive/Data_M/1M_Lem_clusters/km_25',
 '/content/drive/MyDrive/Data_M/1M_Lem_clusters/km_100',
 '/content/drive/MyDrive/Data_M/1M_Lem_clusters/som_100',
 '/content/drive/MyDrive/Data_M/1M_Lem_clusters/som_225']

In [9]:
liste_clean , liste_brut = make_liste(list_file)

In [10]:
list_file

['/content/drive/MyDrive/Data_M/1M_Lem_clusters/km_25',
 '/content/drive/MyDrive/Data_M/1M_Lem_clusters/km_50',
 '/content/drive/MyDrive/Data_M/1M_Lem_clusters/km_100',
 '/content/drive/MyDrive/Data_M/1M_Lem_clusters/som_100',
 '/content/drive/MyDrive/Data_M/1M_Lem_clusters/som_225',
 '/content/drive/MyDrive/Data_M/1M_Lem_clusters/som_400']

In [11]:
%%time
clean_cluster= matrice(liste_clean)

CPU times: user 1.89 s, sys: 673 ms, total: 2.56 s
Wall time: 4.48 s


In [12]:
%%time
brut_cluster= matrice(liste_brut)

CPU times: user 3.89 s, sys: 1.07 s, total: 4.96 s
Wall time: 6.7 s


In [13]:
data = make_df(brut_cluster, clean_cluster)

In [14]:
data[1][0].head()

,tweet,clean_twt
0,Meufff😻\n,meufff\n
1,😂😂😂 il ont peur de quoi? Est-ce qu'on mange le...,peur?-ce qu'mange?\n
2,Y'a pas à me bloquer comme ça ???? https://t.c...,bloquer????\n
3,Ah bon🧍\n,\n
4,"Pst, pst, ilustradores 👇 https://t.co/RHSqZv6g...","pst, pst, ilustradore\n"


data_km : 
1.   index 0 : km25
2.   index 1: km50
3.   index 2: km100

data_SOM : 
1.   index 3 : som100
2.   index 4: som225
3.   index 5: som400

In [ ]:
!pip install gensim==4.0
!pip install python-Levenshtein

In [16]:
from gensim.models import KeyedVectors

In [17]:
# récupérer les embeddings entrainés sur nos données
w2v_lem = KeyedVectors.load('/content/drive/MyDrive/Data_M/w2v/w2vec_model_d300_1M_Lem')

In [18]:
#encode un message
def encode(msg):
    return np.mean([w2v_lem[word] for word in msg if word in w2v_lem] or [np.zeros(300)], axis = 0)

In [19]:
# Encoder une liste de messages 
def encoded(liste):
  w2v=[]
  for i in range(len(liste)):
    w2v.append(np.array([encode(msg.split()) for msg in liste[i]]))
  return w2v

Encodage des différents set de clusters

In [20]:
def liste_encoded(liste_cluster):
  dataset_encoded=[]
  for i in range(len(liste_cluster)):
    dataset_encoded.append(encoded(liste_cluster[i]))
  return dataset_encoded

In [21]:
dataset_encoded= liste_encoded(clean_cluster)

Entrainement des différents set de clusters

In [22]:
from sklearn.ensemble import IsolationForest

# definir le modèle ainsi que ses paramètres
if_model=IsolationForest(n_jobs=-1, random_state=42)


Créer une matrice qui stock les différents models d'IF correspondant aux différents clusters

In [23]:
def train_if (dataset):
  models=[[] for _ in range(len(dataset))]
  for i in range(len(dataset)):
    for n in range(len(dataset[i])):
      if_model=IsolationForest(n_jobs=-1, random_state=42)
      models[i].append(if_model.fit(dataset[i][n]))
  return models

Appliquer IF sur les clusters de Km

In [24]:
%%time
models = train_if(dataset_encoded)

CPU times: user 19min 20s, sys: 30.1 s, total: 19min 50s
Wall time: 12min 5s


Appliquer IF sur les clusters de SOM

Prédictions pour les différents modèles

In [25]:
def pred(dataset,model):
  preds=[[] for _ in range(len(dataset))]
  for i in range(len(dataset)):
    for n in range(len(dataset[i])):
      preds[i].append(model[i][n].predict(dataset[i][n]))
  return preds

In [26]:
pred= pred(dataset_encoded, models)

In [27]:
def add_pred_to_df(dataset,pred):
  for i in range(len(pred)):
    for n in range(len(pred[i])):
      dataset[i][n]['pred']= pred[i][n]

In [28]:
add_pred_to_df(data, pred)

In [29]:
len(data[5])

400

In [ ]:
data[0][11]

In [31]:
# Trouver les outliers de chaque cluster
# ceux dont l'index est -1
def outliers(df):
  outliers=[]
  for i in range(len(df)):
    outliers.append(df[i]['tweet'].loc[df[i]['pred']==-1])
  return outliers

In [32]:
liste_cluster=["km25","km50","km100","som100","som225","som400"]

In [44]:
def save_outliers(outliers,string):
  for i in range(len(outliers)):
    with open(f"drive/MyDrive/lem/"+string+f"/outliers_{i}", 'w') as f:
      for line in outliers[i]:
        f.write(line)

In [34]:
def loop_outliers(data, liste):
  for i in range(len(data)):
    save_outliers(outliers(data[i]),liste[i])

In [45]:
loop_outliers(data, liste_cluster)